# Direct join between French Crop Usage (FCU) and TAXREF-LD

Join condition: FCU crop name = TAXREF-LD vernacular name

### Initializations

In [15]:
import json
import os
from string import Template
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, POST
from time import sleep
from math import isnan, nan

In [16]:
# Initializations
prefixes = '''
prefix fcu:            <http://ontology.inrae.fr/frenchcropusage/>
prefix owl:            <http://www.w3.org/2002/07/owl#>
prefix rdf:            <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs:           <http://www.w3.org/2000/01/rdf-schema#>
prefix skos:           <http://www.w3.org/2004/02/skos/core#>
prefix skosxl:         <http://www.w3.org/2008/05/skos-xl#>
prefix taxrefp:        <http://taxref.mnhn.fr/lod/property/>
prefix taxrefrk:       <http://taxref.mnhn.fr/lod/taxrank/>
prefix xsd:            <http://www.w3.org/2001/XMLSchema#>
'''

In [17]:
fcu_endpoint = "http://ontology.inrae.fr/frenchcropusage/sparql"
#fcu_endpoint = "http://localhost:8080/sparql"
taxref_endpoint = "https://taxref.mnhn.fr/sparql"

In [18]:
def exec_sparql(endpoint, query):
    """
    Execute a SPARQL query and return results as a Pandas DataFrame.
    Invocation uses the HTTP POST method. Requested result format is JSON.

    In case of failure, the function retries up to MAX_ATTEMPTS attempts while waiting increasing time between each attempt.
    An empty DataFrame is returned in case no attempt is successful.
    """
    # Max number of time a SPARQL query can fail before giving up
    MAX_ATTEMPTS = 3

    sparql = SPARQLWrapper(endpoint)
    sparql.setMethod(POST)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    
    attempt = 1;
    success = False
    while not success and attempt <= MAX_ATTEMPTS:
        try:
            raw_results = sparql.query()
            results = raw_results.convert()
        except:
            print(f'Error while executing SPARQL query (attempt {attempt}/{MAX_ATTEMPTS}).', end=' ')
            if attempt < MAX_ATTEMPTS:
                # Wait a few seconds before next attempt (5, 10, 15, ...)
                print(f'Will retry in {attempt * 5}s.')
                sleep(attempt * 5)
        else:
            success = True
        finally:
            attempt = attempt + 1

    if success:
        if 'head' in results and 'results' in results:
            cols = results['head']['vars']
            out = []
            for row in results['results']['bindings']:
                item = []
                for c in cols:
                    item.append(row.get(c, {}).get('value'))
                out.append(item)
            return pd.DataFrame(out, columns=cols)
        else:
            print('Invalid SPARQL result. Will return empty DataFrame.\n' + str(results))
    else:
        print(f'Unable to execute SPARQL query after {MAX_ATTEMPTS} attempts. Will return empty DataFarme.')
                  
    return pd.DataFrame()            


def dataframe_preview(df, start=0, end=10):
    print("== Number of lines: " + str(df.shape[0]))
    print("== Number of unique values:")
    print(df.nunique())
    display(df[start:end])

___
# Get all varieties from FCU
FCU labels = vernacular names

In [19]:
query =  prefixes + '''
select distinct ?fcu_concept ?fcu_name ?fcu_name_type where {

    # Select only cultivated crops
    <http://ontology.inrae.fr/frenchcropusage/Usages_plantes_cultivees> skos:narrower+ ?fcu_concept.

    # Keep only leaves or their direct parent but not above
    FILTER (NOT EXISTS { ?fcu_concept skos:narrower/skos:narrower ?child. })

    # Get prefered and alternate labels and keep track of the type of label
    ?fcu_concept a skos:Concept.
    { ?fcu_concept skos:prefLabel ?lb. bind("pref" as ?fcu_name_type) }
    UNION
    { ?fcu_concept skos:altLabel  ?lb. bind("alt" as ?fcu_name_type) }

    bind(str(lcase(?lb)) as ?fcu_name)

} order by ?fcu_concept
'''

In [20]:
%time df_fcu = exec_sparql(fcu_endpoint, query)

Wall time: 129 ms


In [21]:
dataframe_preview(df_fcu, end=5)

== Number of lines: 1515
== Number of unique values:
fcu_concept       446
fcu_name         1481
fcu_name_type       2
dtype: int64


,fcu_concept,fcu_name,fcu_name_type
0,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref
1,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot,alt
2,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier pays,pref
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot pays,alt
4,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier des antilles,alt


---
# Join FCU with TAXREF-LD
### FCU varieties names = TAXREF-LD vernacular names

The SPARQL endpoint of TAXREF-LD does not support passing all varietes at once in a VALUES clause (ends up with an "HTTP Error 502: Proxy Error").
To avoid this, we make a SPARQL query for each FCU variety. It takes more time but it completes.

In [23]:
queryTpl = Template(prefixes + '''
select distinct ("$fcu_concept" as ?fcu_concept) ("$fcu_name" as ?fcu_name) ("$fcu_name_type" as ?fcu_name_type) ?taxref_full_name ?taxon ?rank
from <http://taxref.mnhn.fr/lod/graph/classes/15.0>
from <http://taxref.mnhn.fr/lod/graph/vernacular/15.0>
from <http://taxref.mnhn.fr/lod/graph/concepts>
where {
    ?taxon
       a                      owl:Class;
       taxrefp:hasReferenceName [ rdfs:label ?taxref_full_name ];
       taxrefp:hasRank        ?rank;
       taxrefp:vernacularName ?vn.

    # To lowercase + remove language tag if any
    filter (str(lcase(?vn)) = "$fcu_name")
    
    # All ranks up to spcecies but not above
    filter (?rank in (
        taxrefrk:Species,  taxrefrk:SemiSpecies, taxrefrk:MicroSpecies, taxrefrk:SubSpecies, taxrefrk:Natio, 
        taxrefrk:Varietas, taxrefrk:SubVarietas, taxrefrk:Forma,        taxrefrk:SubForma,   taxrefrk:FormaSpecies,
        taxrefrk:Linea,    taxrefrk:Clone,       taxrefrk:Race,         taxrefrk:Cultivar,   taxrefrk:Morpha,
        taxrefrk:Abberatio ))
}
''')

In [26]:
# Set a max number of queries to submit. 0 = unlimited.
MAX_QUERIES = 0

# Result DataFrame
df_fcu_taxref = pd.DataFrame()

for idx, row in df_fcu.iterrows():
    query = queryTpl.substitute(
        fcu_concept = row['fcu_concept'].strip(), 
        fcu_name = row['fcu_name'].strip().lower(),
        fcu_name_type = row['fcu_name_type'].strip().lower())
    #print(query)
    
    print(f"---- Running query {idx + 1}/{len(df_fcu)} - name = {row['fcu_name']}")
    %time _df = exec_sparql(taxref_endpoint, query)
    print(f'Number of results: {_df.shape[0]}')
    df_fcu_taxref = df_fcu_taxref.append(_df)
    
    # Keep track of the names not matched with TAXREF
    if _df.shape[0] == 0:
        nomatch_row = {'fcu_concept': row['fcu_concept'].strip(), 'fcu_name': row['fcu_name'].strip().lower(), 'fcu_name_type': row['fcu_name_type'].strip().lower(), 'taxref_full_name': None, 'taxon': None, 'rank': None}
        df_fcu_taxref = df_fcu_taxref.append(nomatch_row, ignore_index=True)

    if MAX_QUERIES > 0 and (idx+1) >= MAX_QUERIES:
        break

---- Running query 1/1515 - name = abricotier
Wall time: 817 ms
Number of results: 0
---- Running query 2/1515 - name = abricot
Wall time: 799 ms
Number of results: 0
---- Running query 3/1515 - name = abricotier pays
Wall time: 804 ms
Number of results: 0
---- Running query 4/1515 - name = abricot pays
Wall time: 815 ms
Number of results: 0
---- Running query 5/1515 - name = abricotier des antilles
Wall time: 750 ms
Number of results: 0
---- Running query 6/1515 - name = mamey
Wall time: 599 ms
Number of results: 0
---- Running query 7/1515 - name = abricotier-pays
Wall time: 763 ms
Number of results: 0
---- Running query 8/1515 - name = actinidia
Wall time: 786 ms
Number of results: 0
---- Running query 9/1515 - name = groseille de chine
Wall time: 771 ms
Number of results: 0
---- Running query 10/1515 - name = kiwi
Wall time: 716 ms
Number of results: 0
---- Running query 11/1515 - name = yang tao
Wall time: 696 ms
Number of results: 0
---- Running query 12/1515 - name = actinidier


In [50]:
# Number of matches
df_fcu_taxref_matched = df_fcu_taxref.dropna()
dataframe_preview(df_fcu_taxref_matched)

== Number of lines: 393
== Number of unique values:
fcu_concept         161
fcu_name            185
fcu_name_type         2
taxref_full_name    369
taxon               369
rank                  4
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,taxref_full_name,taxon,rank
26,http://ontology.inrae.fr/frenchcropusage/Ananas,ananas,pref,"Ananas comosus (L.) Merr., 1917",http://taxref.mnhn.fr/lod/taxon/447782,http://taxref.mnhn.fr/lod/taxrank/Species
27,http://ontology.inrae.fr/frenchcropusage/Anemones,anémone,pref,Anemone hortensis nothosubsp. fulgens (J.Gay) ...,http://taxref.mnhn.fr/lod/taxon/131396,http://taxref.mnhn.fr/lod/taxrank/SubSpecies
31,http://ontology.inrae.fr/frenchcropusage/Angel...,angélique,pref,"Dicorynia guianensis Amshoff, 1939",http://taxref.mnhn.fr/lod/taxon/733639,http://taxref.mnhn.fr/lod/taxrank/Species
37,http://ontology.inrae.fr/frenchcropusage/Arach...,arachide,pref,"Arachis hypogaea L., 1753",http://taxref.mnhn.fr/lod/taxon/611649,http://taxref.mnhn.fr/lod/taxrank/Species
61,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,"Cynara cardunculus L., 1753",http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
64,http://ontology.inrae.fr/frenchcropusage/Asperges,asperge blanche,alt,"Asparagus albus L., 1753",http://taxref.mnhn.fr/lod/taxon/84265,http://taxref.mnhn.fr/lod/taxrank/Species
70,http://ontology.inrae.fr/frenchcropusage/Attiers,pomme cannelle,alt,"Annona squamosa L., 1753",http://taxref.mnhn.fr/lod/taxon/446901,http://taxref.mnhn.fr/lod/taxrank/Species
72,http://ontology.inrae.fr/frenchcropusage/Auber...,aubergine,pref,"Solanum melongena L., 1753",http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
73,http://ontology.inrae.fr/frenchcropusage/Avoca...,avocatier,pref,"Persea americana Mill., 1768",http://taxref.mnhn.fr/lod/taxon/447273,http://taxref.mnhn.fr/lod/taxrank/Species
77,http://ontology.inrae.fr/frenchcropusage/Avoines,avoine cultivée,alt,"Avena sativa subsp. sativa L., 1753",http://taxref.mnhn.fr/lod/taxon/132016,http://taxref.mnhn.fr/lod/taxrank/SubSpecies


### Matches:
- FCU: 161 unique concepts, 185 unique labels
- TAXREF-LD: 369 unique taxa from 4 ranks

### Exports

In [36]:
df_fcu_taxref.to_excel("result_fcu_taxref.xlsx")

In [51]:
# Reshape the DataFrame for later merging with other methods
df_fcu_taxref_matched.drop(columns=['fcu_name_type', 'fcu_name'], inplace=True)
df_fcu_taxref_matched.rename(columns={'taxref_full_name': 'taxref_ref_full_name'}, inplace=True)
df_fcu_taxref_matched.insert(1, 'method', 'direct')
df_fcu_taxref_matched.insert(2, 'geves_name', '')
df_fcu_taxref_matched.insert(3, 'eppo_scientific_name', '')

C:\Users\fmichel\anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\fmichel\anaconda3\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [52]:
dataframe_preview(df_fcu_taxref_matched)
df_fcu_taxref_matched.to_excel("result_fcu_taxref_merge.xlsx", index=False)
df_fcu_taxref_matched.to_csv("result_fcu_taxref_merge.csv", index=False)

== Number of lines: 393
== Number of unique values:
fcu_concept             161
method                    1
geves_name                1
eppo_scientific_name      1
taxref_ref_full_name    369
taxon                   369
rank                      4
dtype: int64


,fcu_concept,method,geves_name,eppo_scientific_name,taxref_ref_full_name,taxon,rank
26,http://ontology.inrae.fr/frenchcropusage/Ananas,direct,,,"Ananas comosus (L.) Merr., 1917",http://taxref.mnhn.fr/lod/taxon/447782,http://taxref.mnhn.fr/lod/taxrank/Species
27,http://ontology.inrae.fr/frenchcropusage/Anemones,direct,,,Anemone hortensis nothosubsp. fulgens (J.Gay) ...,http://taxref.mnhn.fr/lod/taxon/131396,http://taxref.mnhn.fr/lod/taxrank/SubSpecies
31,http://ontology.inrae.fr/frenchcropusage/Angel...,direct,,,"Dicorynia guianensis Amshoff, 1939",http://taxref.mnhn.fr/lod/taxon/733639,http://taxref.mnhn.fr/lod/taxrank/Species
37,http://ontology.inrae.fr/frenchcropusage/Arach...,direct,,,"Arachis hypogaea L., 1753",http://taxref.mnhn.fr/lod/taxon/611649,http://taxref.mnhn.fr/lod/taxrank/Species
61,http://ontology.inrae.fr/frenchcropusage/Artic...,direct,,,"Cynara cardunculus L., 1753",http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
64,http://ontology.inrae.fr/frenchcropusage/Asperges,direct,,,"Asparagus albus L., 1753",http://taxref.mnhn.fr/lod/taxon/84265,http://taxref.mnhn.fr/lod/taxrank/Species
70,http://ontology.inrae.fr/frenchcropusage/Attiers,direct,,,"Annona squamosa L., 1753",http://taxref.mnhn.fr/lod/taxon/446901,http://taxref.mnhn.fr/lod/taxrank/Species
72,http://ontology.inrae.fr/frenchcropusage/Auber...,direct,,,"Solanum melongena L., 1753",http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
73,http://ontology.inrae.fr/frenchcropusage/Avoca...,direct,,,"Persea americana Mill., 1768",http://taxref.mnhn.fr/lod/taxon/447273,http://taxref.mnhn.fr/lod/taxrank/Species
77,http://ontology.inrae.fr/frenchcropusage/Avoines,direct,,,"Avena sativa subsp. sativa L., 1753",http://taxref.mnhn.fr/lod/taxon/132016,http://taxref.mnhn.fr/lod/taxrank/SubSpecies
